In [11]:
from utils.aqi_calculation import AqiCalculation
from utils.parsing_table import ParsingTable
from datetime import date
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None

path = '../resources/aqi_breakpoints.xlsx'
aqi = AqiCalculation(dict_df=pd.read_excel(path, sheet_name=['electrosmog_lf', 'wifi_level',
                                                             'temperature', 'humidity', 'air_pressure',
                                                             'ambient_light', 'tvoc',
                                                             'co2', 'co2e', 'pm10', 'pm25', 'sound',
                                                             'electrosmog_hf']))
arpa_pool = {}
arpa_path = '../resources/arpa/'
arpa_list = os.listdir(arpa_path)
for file in arpa_list:
    if file.endswith('.txt'):
        temp: pd.DataFrame()
        pm = str(file.split('_')[1].replace('.', '').lower() + '_ext')
        tempT:str
        print(pm.find('pm25'))
        if pm=='pm25_ext':
            tempT='3'
        else:
            tempT='5'

        temp = pd.read_table(arpa_path + file, delimiter='\s+')[['anno', 'jd', tempT]]
        temp.rename(columns={tempT: pm}, inplace=True)
        temp['date'] = pd.to_datetime(temp['anno'].astype(str) + temp['jd'].astype(str), format='%Y%j').dt.date
        temp= temp[['date',pm]]
        print(f'{file.split("_")[2]}  {pm}')
        try:
            arpa_pool[pm] = pd.concat([temp, arpa_pool[pm]])
        except:
            arpa_pool[pm] = temp

arpa_pool= arpa_pool['pm10_ext'].merge(arpa_pool['pm25_ext'],on='date')
arpa_pool['date']= pd.to_datetime(arpa_pool['date'])

meteo = pd.read_csv('../resources/meteo.csv').rename(
    columns={'DATA': 'date', 'TMEDIA °C': 'ext_temperature', 'UMIDITA %': 'ext_humidity', 'VENTOMEDIA km/h': 'ext_wind',
             'PRESSIONESLM mb': 'ext_pressure', 'FENOMENI': 'ext_weather'})[
    ['date', 'ext_temperature', 'ext_humidity', 'ext_wind', 'ext_pressure', 'ext_weather']]
meteo['ext_weather'] = meteo['ext_weather'].fillna('sole')
meteo['date'] = pd.to_datetime(meteo['date'], format="%d/%m/%Y")

pool = {}
pool_by_day = {}
pt = ParsingTable()
input_path = '../resources/semi_clean_file/'
dir_list = os.listdir(input_path)
for file in dir_list:
    if file.endswith('.csv'):
        temp: pd.DataFrame
        device_id: int
        naming = file.split('.')[0]
        print(naming)
        temp = pt.parse(input_path=input_path + file)
        temp['date'] = pd.to_datetime(temp['date'])
        pool[naming] = temp
        device_id = temp['device_id'].iloc[0]
        temp = temp.groupby(temp['date'].dt.date).agg({'iaq': 'mean', 'electrosmog_lf': 'mean', 'wifi_level': 'mean',
                                                       'wifi_n': 'mean', 'temperature': 'mean', 'humidity': 'mean',
                                                       'air_pressure': 'mean', 'ambient_light': 'mean', 'tvoc': 'mean',
                                                       'co2': 'mean', 'co2e': 'mean', 'pm10': 'mean', 'pm25': 'mean',
                                                       'sound': 'mean', 'electrosmog_hf': 'mean'}).reset_index()
        temp['device_id'] = device_id
        temp['date'] = pd.to_datetime(temp['date'])
        temp = temp.merge(meteo, on='date')
        temp = temp.merge(arpa_pool,on='date')
        pool_by_day[naming] = temp

df_arpa = pd.read_csv('../resources/dati_nrt_informambiente_mv.csv')

print('end loading')

-1
2019  pm10_ext
-1
2020  pm10_ext
-1
2021  pm10_ext
-1
2022  pm10_ext
0
2019  pm25_ext
0
2020  pm25_ext
0
2021  pm25_ext
0
2022  pm25_ext
CAP153_1


C:\Users\lucab\Documents\Python\Assignement_RBS\src\utils\parsing_table.py:127: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(path),


CAP153_2
CAP153_3


C:\Users\lucab\Documents\Python\Assignement_RBS\src\utils\parsing_table.py:127: DtypeWarning: Columns (13,14,15,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(path),


CAP186_1
CAP186_2


C:\Users\lucab\Documents\Python\Assignement_RBS\src\utils\parsing_table.py:127: DtypeWarning: Columns (13,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(path),


CAP186_3
end loading


In [12]:
pool_by_day['CAP153_1']

,date,iaq,electrosmog_lf,wifi_level,wifi_n,temperature,humidity,air_pressure,ambient_light,tvoc,...,sound,electrosmog_hf,device_id,ext_temperature,ext_humidity,ext_wind,ext_pressure,ext_weather,pm10_ext,pm25_ext
0,2020-01-29,35.113333,162.000000,-64.600000,6.333333,21.393333,46.300000,1011.660000,381.053333,16.466667,...,53.253333,0.286667,219,11.0,82.0,8.0,1014.0,pioggia,43.0,13.0
1,2020-01-31,133.361212,183.515152,-32.696970,22.521212,25.269091,37.952727,1020.461212,200.735758,11.448485,...,52.449697,1.469091,219,12.0,75.0,8.0,1021.0,sole,56.0,24.0
2,2020-02-01,71.595533,172.536082,-31.037801,24.285223,25.195533,41.262543,1020.073540,186.563918,12.639175,...,48.324742,1.427835,219,12.0,86.0,6.0,1021.0,sole,62.0,44.0
3,2020-02-02,73.602759,174.189655,-31.144828,23.913793,25.114138,43.229655,1020.692759,187.672069,6.620690,...,46.879655,1.406207,219,13.0,81.0,7.0,1022.0,sole,37.0,27.0
4,2020-02-03,125.578621,174.344828,-32.558621,22.658621,25.902759,40.904828,1018.382414,207.726552,16.113793,...,50.207241,1.514828,219,13.0,81.0,7.0,1019.0,sole,37.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,2022-10-29,81.217123,173.068493,-43.883562,18.880137,24.854452,47.390068,1022.786301,132.606507,29.315068,...,42.829452,1.509589,219,18.0,68.0,6.0,1025.0,sole,30.0,18.0
842,2022-10-30,129.048013,172.645695,-43.645695,18.473510,24.159934,46.766556,1020.713907,21.128808,38.274834,...,41.748675,1.464238,219,17.0,67.0,6.0,1022.0,sole,18.0,13.0
843,2022-10-31,110.533448,171.837931,-42.362069,18.234483,23.575862,45.921379,1020.797931,141.697241,34.727586,...,45.764828,1.533793,219,18.0,58.0,5.0,1022.0,sole,24.0,14.0
844,2022-11-01,138.262414,173.855172,-43.482759,18.727586,23.618966,48.979655,1021.795172,20.078966,56.834483,...,42.562414,1.535172,219,17.0,69.0,5.0,1023.0,sole,23.0,16.0


In [13]:
clean_file_153 = pd.concat([pool['CAP153_1'], pool['CAP153_2'], pool['CAP153_3']])
clean_file_153.to_csv('../resources/concat_files/all_153.csv')

clean_file_186 = pd.concat([pool['CAP186_1'], pool['CAP186_2'], pool['CAP186_3']])
clean_file_186.to_csv('../resources/concat_files/all_186.csv')

clean_file_153_meteo = pd.concat([pool_by_day['CAP153_1'], pool_by_day['CAP153_2'], pool_by_day['CAP153_3']])
clean_file_153_meteo.to_csv('../resources/concat_files/all_pm_meteo_153.csv')

clean_file_186_meteo = pd.concat([pool_by_day['CAP186_1'], pool_by_day['CAP186_2'], pool_by_day['CAP186_3']])
clean_file_186_meteo.to_csv('../resources/concat_files/all_pm_meteo_186.csv')





In [ ]:
test = pool['CAP186_2'].copy()

test = test[(test['battery'] != 'good') & (test['battery'] != 'low')]

test

In [ ]:
device_number: pd.DataFrame = pd.DataFrame()
device_number = device_number.assign(device_id=pool_33_3['device_id'],
                                     device_n=pool_33_3.groupby('device_id')['device_id'].transform(
                                         'count')).drop_duplicates()
# pool_33_3['dev_n'] = pool_33_3.groupby('device_id')['device_id'].transform('count')
device_number

In [ ]:
# oaq --> iaq (con stesse variabili conosciute) ==> troviamo efficienza filtraggio interno